# This is the most stupid LLM.....

## Imports

In [131]:
import os
import json
import ollama
import random
import re

## Ollama Functions

In [132]:
def think(str):
  response = ollama.chat(model='mistral', messages=[
    {
      'role': 'user',
      'content': f'{str}',
    },
  ])
  return response['message']['content'] #type: ignore

## Variables

In [133]:
# Variables
data_directory = "data"
convo = "convo.json"
info = "info.json"

# Paths
convo_path = data_directory + "/" + convo
info_path = data_directory + "/" + info

characters_to_remove = "[];!?\\/\"' "

## Prompts

In [134]:
def interaction_rules(str):
  blob = f'''
  ## Take note
  - it must not have numbers
  - it is not listed numbers, I only need 1
  - english only
  - if it is a question, include question mark '?'
  - Do not include the [your name] part  
  - if it is a listed with numbers, like for example:
    1. data1
    2. data2
  - please remove the numbers. and make it numberless instead:
    [data1], [data2]
  - each ${str} must be contain on a square brackets
  - NEVER FORGET THE [SQUARE BRACKETS] AND "," FOR DIVISION
  - do not use special characters like slsh and backslash, and double quote
  '''
  return blob

#### Greetings

In [135]:
greetings_word = f'''
Provide me a 5 list words that can initiate a greetings, only 5 words outcome
for example:
[hi], [hello], [hello there]
{interaction_rules("words")}
'''

In [136]:
greetings_phrase = f'''
Provide me a 5 list words or phrase that can initiate a greetings, only 5 outcome
for example:
[hello there],[hi there],[howdy]

{interaction_rules("phrase or word")}
'''

In [137]:
greetings_sentence = f'''
Provide me a 5 list of sentences or questions that can initiate a greetings, only 5 outcome
for example:
[how are you doing today?],[how are you?]

{interaction_rules("sentences or questions")}
'''

## File Initialization Dependencies

In [138]:
# Ensures that the data are present....
def mkjson(filename):
    if not os.path.exists(filename):
        with open(filename, 'w') as file:
            pass
if not os.path.exists(data_directory):
    os.makedirs(data_directory)

mkjson(convo_path)
mkjson(info_path)

## JSON Dataset

### Template

In [139]:
def is_file_blank(file_path):
    if os.path.exists(file_path):
        if os.path.getsize(file_path) >= 1:
            return False  # File is not blank
        else:
            return True  # File is blank
    else:
        return None  # File does not exist

In [140]:
if is_file_blank(convo_path):
  data = {
    "questions": []
  }
  with open(convo_path, "w") as json_file:
    json.dump(data, json_file)

### JSON Functions

In [141]:
def load_json(file_path):
    with open(file_path, 'r') as file:
        return json.load(file)

In [142]:
def write_json(file_path, data):
    with open(file_path, 'w') as file:
        json.dump(data, file, indent=4)

In [143]:
def ask(str):
    # Function to load JSON from file
    def load_json(file_path):
        with open(file_path, 'r') as file:
            return json.load(file)

    # Function to write JSON to file
    def write_json(file_path, data):
        with open(file_path, 'w') as file:
            file.write(data)

    data = load_json(convo_path)
    questions = data['questions']
    
    # Check if the provided string is already present in any existing question
    str_exists = any(str in q['question'] for q in questions)

    # If the string is not present, add it as a new question
    if not str_exists:
        data['questions'].append({"question": str, "response": []})
        transformed_json = json.dumps(data, indent=2)
        write_json(convo_path, transformed_json)
        print(transformed_json)
    else:
        print(f"The question '{str}' already exists in the conversation data.")

## User Questions Prediction Mining

### Question Mining

In [144]:
def question_juiced_up():
  promps = [
    greetings_word,
    greetings_phrase,
    greetings_sentence
  ]
  random_number = random.randint(0, len(promps)-1)
  inserto = promps[random_number]
  print("--> inserto: ", inserto)
  qm = think(inserto)
  qm_arr = re.findall(r'\[([^\]]+)\]', qm)
  for j in qm_arr:
    ask(j)
  
try:
  question_juiced_up()
except:
  print("Output Error...")

--> inserto:  
Provide me a 5 list of sentences or questions that can initiate a greetings, only 5 outcome
for example:
[how are you doing today?],[how are you?]


  ## Take note
  - it must not have numbers
  - it is not listed numbers, I only need 1
  - english only
  - if it is a question, include question mark '?'
  - Do not include the [your name] part  
  - if it is a listed with numbers, like for example:
    1. data1
    2. data2
  - please remove the numbers. and make it numberless instead:
    [data1], [data2]
  - each $sentences or questions must be contain on a square brackets
  - NEVER FORGET THE [SQUARE BRACKETS] AND "," FOR DIVISION
  - do not use special characters like slsh and backslash, and double quote
  

{
  "questions": [
    {
      "question": "Hello",
      "response": [
        " Hello! How can I help you today? Is there something specific you'd like to ask or discuss? I'm here to provide information and answer any questions you may have to the best of my abi

### Validation Mining

In [145]:
def question_validations():
    datos = load_json(convo_path)
    questions = datos["questions"][:]

    questions_to_remove = []
    for question in questions:
        if '"' in question["question"]:
            questions_to_remove.append(question)

    for question in questions_to_remove:
        datos["questions"].remove(question)

    write_json(convo_path, datos)
    
try:
    question_validations()
except:
    print("Output Error...")

### Answering Questions

In [146]:
def answer_it(str):
  blob = f'''
  Pretend as a human, think like a human, emulate human mind, and answer the question
  as polite, and simple as possible, base from the question.
  [Question]> {str}
  
  Take note:
  - Do not say something promising.
  - Answer as humble as possible
  - Just say a good answer
  - If ask about who are you, or what are you you answers RAI, resilient artificial intelligence
  '''
  return blob

In [147]:
datos = load_json(convo_path)
questions = datos['questions']

def answer_data(index, data):
  for question in questions:
      if question["question"] == index:
          if data not in question["response"]:
              question["response"].append(data)
              
cube = [item["question"] for item in datos['questions']]
for j in cube:
  datax = think(answer_it(j))
  print(f"The question {j} has been answered")
  answer_data(j, datax)
  
  write_json(convo_path, datos)

The question Hello has been answered
The question Hey has been answered
The question Greetings has been answered
The question Hi has been answered
The question Morning has been answered
The question Afternoon has been answered
The question Evening has been answered
The question Goodday has been answered
The question Good morning has been answered
The question Good afternoon has been answered
The question Good evening has been answered
The question Hey there has been answered
The question How are you? has been answered
The question Nice day has been answered
The question Hello, I'm here has been answered
The question Welcome back has been answered
The question Gidday has been answered
The question How do you do? has been answered
